In [1]:
from docplex.mp.model import Model  #导出库，只用这一个就够了
import matplotlib.pyplot as plt#选取了用户47的365天的数据
import numpy as np
import pandas as pd
import csv
import cplex
from cplex.exceptions import CplexError
import time
import math
import openpyxl
from openpyxl import Workbook
import xlrd
import torch
from torch import nn
#from matplotlib.animation import FuncAnimation
#import matplotlib.animation as animation
import os

In [2]:
def Read_year(year,user):
    #read file, for example
    file_path = 'C:/Users/chenxihui/Desktop/code/project/'+str(year)+'.csv'
    file = open(file_path)
    data = []
    for i in file.readlines():
        data.append(i)
    file.close()
    new_data = []

    for row in data:
        tmp = row.strip('\n')  #去掉每行最后的回车符
        tmp = tmp.split(',')   #根据','来分割字符串，使之成为含有一个个数据的列表
        new_data.append(tmp)   #new_data的每一行数据就是一个列表
    select_data = []
    position=[1,3,6,12,14,15,16,18,19,22,23,26,28,31,32,34,38,41,42,43,45,46,47]
    #这是在cluster里分类产生的结果，是人数最多的类，以此为对象
    users=['2','13','14','20','33','35','38','39','56','69','73','74','75'
          ,'82','87','88','101','104','106','109','110','119','124','130'
          ,'137','141','144','152','153','157','161','169','176','184','188'
          ,'189','193','201','202','204','206','207','210','211','212','214'
          ,'218','244','246','253','256','273','276','297']
    
    """
        user_chosen=[]
    for item in position:
        user_chosen.append(users[item])
    """
        
    length=len(new_data)
    for i in range(length):
        if(new_data[i][0]==users[user]):
            select_data.append(new_data[i][3:])#[5:]
            pv_size=float(new_data[i][1])
    GC=[]
    CL=[]
    GG=[]
    Length_select_data = len(select_data)
    for i in range(Length_select_data):
        if(select_data[i][0]=='GC'):
            GC.append(select_data[i][2:])
        elif(select_data[i][0]=='CL'):
            CL.append(select_data[i][2:])
        else:
            GG.append(select_data[i][2:])
            
    Total_Load=[]
    temp=[]
    a=0.0
    for i in range(Length_select_data):
        if(select_data[i][0]=='GC' and select_data[i+1][0]=='CL'):
            for j in range(len(select_data[i][2:])):
                a=pd.to_numeric(select_data[i][j+2])+pd.to_numeric(select_data[i+1][j+2])
                temp.append(a)
            Total_Load.append(temp)
        elif(select_data[i][0]=='GC' and select_data[i+1][0]!='CL'):
            Total_Load.append(select_data[i][2:])
        temp=[]
        
    GC_temp=np.array(GC)
    GC_array=[]
    GC_array = GC_temp.astype(np.float)

    GG_temp=np.array(GG)
    GG_array=[]
    GG_array = GG_temp.astype(np.float)*2
    
    Total_Load_temp=np.array(Total_Load)
    Total_Load_array=[]
    Total_Load_array = Total_Load_temp.astype(np.float32)*2#功率
    
    return GG_array,Total_Load_array,pv_size

In [3]:
def milp_one2(T_tou,T_fit,Total_Load_array,GG_array,bat_state,i,d,pb_bar,bat_max,efficiency_i,efficiency_c,dh,pg_bar):
    expression=0
    tfit=[]
    model = Model()####对变量命名并且给出上下界
    var_list = np.arange(0,48,1)
    X1 = model.continuous_var_list(var_list,lb=0.0,ub=pg_bar,name='X1')#代表pg_import(pg+)，功率
    X2 = model.continuous_var_list(var_list,lb=0.0,ub=pg_bar,name='X2')#代表pg_export(pg-)
    Y1 = model.continuous_var_list(var_list,lb=0.0,ub=pb_bar,name='Y1')#代表充电功率（pb+）
    Y2 = model.continuous_var_list(var_list,lb=0.0,ub=pb_bar,name='Y2')#代表放电功率（pb-）
    dg=model.binary_var_list(var_list,name='dg')#决策变量电网电力流向（0：用户到电网）
    sb=model.binary_var_list(var_list,name='sb')#决策变量电池充电状态（0：放电）
    eb=model.continuous_var_list(var_list,lb=0,name='eb')#电池容量SOC
    tou=[]
    tou.extend(T_tou[i:])
    tou.extend(T_tou[0:i])#输入分时电价
    
    for j in range(48):
        #tfit.append(T_fit)
        expression+=tou[j]*X1[j]*dh#X1为pg+,X2为pg-,Y1为pb+,Y2为pb-
        expression-=T_fit*X2[j]*dh
    #约束条件（2）
        if(j+i<48):
            model.add_constraint(X1[j]-X2[j]-efficiency_i*efficiency_c*Y1[j]+(efficiency_i/efficiency_c)*Y2[j]==
                         Total_Load_array[d][j+i]-efficiency_i*GG_array[d][j+i])
        else:
            model.add_constraint(X1[j]-X2[j]-efficiency_i*efficiency_c*Y1[j]+(efficiency_i/efficiency_c)*Y2[j]==
                         Total_Load_array[d+1][j+i-48]-efficiency_i*GG_array[d+1][j+i-48])
        
    #约束条件（3）
        bat_state.append(bat_state[-1]+dh*efficiency_c*Y1[j]-(dh/efficiency_c)*Y2[j])#约束条件（3）
        eb[j]=bat_state[-1]
        model.add_constraint(eb[j]<=bat_max)#约束条件（12）
        model.add_constraint(eb[j]>=0)#约束条件（12）
    #约束条件（4）
        model.add_constraint(X1[j]-pg_bar*dg[j]<=0)
    #约束条件（5）
        model.add_constraint(X2[j]+pg_bar*dg[j]<=pg_bar)
    #约束条件（6）
        model.add_constraint(Y1[j]-pb_bar*sb[j]<=0)
    #约束条件（7）
        model.add_constraint(Y2[j]+pb_bar*sb[j]<=pb_bar)
    model.minimize(expression)#目标函数    
    sol = model.solve() #输出解
    temp = sol.get_all_values()
    return temp,bat_state[0]

In [4]:
def MILP_365(Total_Load_array,GG_array,user):
    L={0: 4,1: 5,2: 4,3: 4,4: 5,5: 5,6: 3,7: 3,8: 5,9: 5,10: 5,11: 8,12: 10,13: 7,14: 5,
 15: 5,16: 5,17: 4,18: 4,19: 10,20: 6,21: 5,22: 3,23: 4,24: 5,25: 6,26: 4,27: 9,28: 5,29: 9.99,
 30: 3.2,31: 5,32: 4,33: 3,34: 4,35: 5,36: 8,37: 5,38: 3,39: 4,40: 5,41: 6,42: 5,43: 4,
 44: 5,45: 4,46: 4,47: 5,48: 4,49: 6,50: 9,51: 7,52: 5,53: 4}
    days=364
    efficiency_i=0.9
    efficiency_c=0.95
    dh=0.5
    pg_bar=20
    T_flat=0.235018
    T_fit=0.09
    T_tou=[0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,
       0.38588,0.38588, 0.38588,0.38588, 
      0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,
      0.38588,0.38588,0.38588,0.38588,0.38588,0.38588,
      0.37147,0.37147,0.37147,0.37147,
      0.2134,0.2134,0.2134,0.2134]
    if(L[user]==3 or L[user]==4):
        init_state=6.5
        pb_bar=4.2
    elif(L[user]==5 or L[user]==6):
        init_state=9.8
        pb_bar=5
    elif(L[user] in [7,8,9,10]):
        init_state=14
        pb_bar=5
    bat_max=init_state
    bat_state=[0.5*init_state]#要输出的电池状态，这是0时刻的初始状态
    pg_import=[[] for _ in range(days)]
    pg_export=[[] for _ in range(days)]
    pb_c=[[] for _ in range(days)]
    pb_d=[[] for _ in range(days)]
    soc=[[] for _ in range(days)]
    soc[0]=[0.5*bat_max]
    a=time.time()
    for d in np.arange(0,days,1): 
        for i in range(48):
        #print(bat_state)
            temp=[]
            temp,before=milp_one2(T_tou,T_fit,Total_Load_array,GG_array,bat_state,i,d,pb_bar,bat_max,efficiency_i,efficiency_c,dh,pg_bar)
        #print(temp[96])
            if(i==47 and d<days-1):
                soc[d+1].append(abs(before+dh*efficiency_c*temp[96]-(dh/efficiency_c)*temp[144]))
            elif(i<47):     
                soc[d].append(abs(before+dh*efficiency_c*temp[96]-(dh/efficiency_c)*temp[144]))#bat_state保存的是滚动域内48个时刻都要满足电池约束，所以一个遍历完后得
        #重置，而SOC是要输出的值
            bat_state=[]
            bat_state.append(soc[d][-1])
        #d=bat_state[-1]+0.455*a[0]-(0.5/0.91)*a[0]
        #print(bat_state[-1])
            pg_import[d].append(abs(temp[0]))
            pg_export[d].append(abs(temp[48]))
            pb_c[d].append(abs(temp[96]))
            pb_d[d].append(abs(temp[144]))
    return soc,pg_import,pg_export,pb_c,pb_d

In [5]:
def save(data,path):
    wb = Workbook()
    ws = wb.active # 激活 worksheet
    [h, l] = np.array(data).shape  # h为行数，l为列数
    for i in range(h):
        row = []
        for j in range(l):
            row.append(format(data[i][j],'.4f'))
        ws.append(row)
    wb.save(path)

In [6]:
def read_data(road):
    workbook=openpyxl.load_workbook("C:/Users/chenxihui/Desktop/code/project/processed_data/"+road+".xlsx")
    shenames=workbook.sheetnames
    worksheet=workbook.worksheets[0]
    name=worksheet.title 
    rows=worksheet.max_row
    columns=worksheet.max_column
    data_read=[[] for i in range(rows)]
    i=0
    for row in worksheet.rows:
        for cell in row:
            data_read[i].append(cell.value)
        i=i+1
    data_temp=np.array(data_read)
    data_array=[]
    data_array=data_temp.astype(np.float)
    return data_array

In [7]:
def samples(T_tou,soc_y,Total_Load_array,GG_array):
    #此版本为362天的样本数据点集合,从第一天开始算，但是第一天里都是空集合
    T_tou_temp=[]
    for i in range(3):
        T_tou_temp.extend(T_tou)
    days=365    
    data_365=[[[] for _ in range(48)] for _ in range(days)]
    for d in range(days):
        for i in range(48):
            data_temp=[]
            GG_temp=[]
            soc_temp=[]#前48个SOC（包含当前时刻）
            ToU_temp=[]
            load_temp=[]
            if(d<363 and d>0):
                if(i==0):
                    soc_temp=np.hstack((soc_y[d-1][i+1:],soc_y[d][0:i+1]))
                    ToU_temp=np.hstack((T_tou[i:],T_tou))
                    load_temp=np.hstack((Total_Load_array[d][i:],Total_Load_array[d+1]))
                    GG_temp=np.hstack((GG_array[d][i:],GG_array[d+1]))
                elif(i==47):
                    soc_temp=soc_y[d]
                    ToU_temp=np.hstack((T_tou,T_tou))
                #print("ToU_temp:",ToU_temp)
                    load_temp=np.hstack((Total_Load_array[d],Total_Load_array[d+1]))
                    GG_temp=np.hstack((GG_array[d],GG_array[d+1]))
                else:
                    soc_temp=np.hstack((soc_y[d-1][i+1:],soc_y[d][0:i+1]))
                    ToU_temp=np.hstack((T_tou[i:],T_tou,T_tou[:i]))
                    GG_temp=np.hstack((GG_array[d][i:],GG_array[d+1],GG_array[d+2][:i]))
                    load_temp=np.hstack((Total_Load_array[d][i:],Total_Load_array[d+1],Total_Load_array[d+2][:i]))
            data_temp=soc_temp
            data_temp=np.hstack((data_temp,ToU_temp))
            data_temp=np.hstack((data_temp,list(map(float, GG_temp))))
            data_temp=np.hstack((data_temp,load_temp))
            data_365[d][i]=data_temp
    return data_365

In [8]:
def generate_train(year,user):
    T_tou=[0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,
       0.38588,0.38588, 0.38588,0.38588, 
      0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,
      0.38588,0.38588,0.38588,0.38588,0.38588,0.38588,
      0.37147,0.37147,0.37147,0.37147,
      0.2134,0.2134,0.2134,0.2134]
    
    GG_array,Total_Load_array,pv_size=Read_year(year,user)
    soc_y=read_data("soc-"+str(year)+"-"+str(user)+"-")
    data_365=samples(T_tou,soc_y,Total_Load_array,GG_array)
    a=np.array(data_365[1:302]).transpose((1,0,2))
    soc_label=[[]for i in range(360)]
    for i in range(360):
        for j in np.arange(1,48):
            soc_label[i].append(soc_y[i+1][j])
        soc_label[i].append(soc_y[i+2][0])
    return a,soc_label

In [16]:
### 此为300天-340天数据作为同一用户的测试集
def RNN_test(year,user,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func):    
    print("Testing...load model...")
    filepath = "./model/rnn"+str(year)+"-"+str(user)+".model"
    checkpoint = torch.load(filepath)
    rnn.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    '''
    print('Model.state_dict:')
    for param_tensor in rnn.state_dict():
        #打印 key value字典
        print(param_tensor,'\t',rnn.state_dict()[param_tensor].size())
 
    print('Optimizer,s state_dict:')
    for var_name in optimizer.state_dict():
        print(var_name,'\t',optimizer.state_dict()[var_name])
    '''
    rnn.eval()
    
    test,soc_label=generate_test(year,user)
    testloss=0
    with torch.no_grad():
        for step in range(40):
            test_x=torch.from_numpy(np.array(test[:,step,np.newaxis])).float().to(device)
            test_y = torch.from_numpy(np.array(soc_label[step+301])[:,np.newaxis,np.newaxis]).float().to(device)
            predict, h_state = rnn(test_x,h_state)
            loss_test = loss_func(predict, test_y)
            testloss+=loss_test
    print(str(user)+"testloss："+str(testloss))

In [10]:
def generate_test(year,user):
    T_tou=[0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,0.2134,
       0.38588,0.38588, 0.38588,0.38588, 
      0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,0.37147,
      0.38588,0.38588,0.38588,0.38588,0.38588,0.38588,
      0.37147,0.37147,0.37147,0.37147,
      0.2134,0.2134,0.2134,0.2134]
    GG_array,Total_Load_array,pv_size=Read_year(year,user)
    soc_y=read_data("soc-"+str(year)+"-"+str(user)+"-")
    data_365=samples(T_tou,soc_y,Total_Load_array,GG_array)
    
    test=np.array(data_365[302:342]).transpose((1,0,2))#测试集从第302天开始
    
    soc_label=[[]for i in range(360)]
    for i in range(360):
        for j in np.arange(1,48):
            soc_label[i].append(soc_y[i+1][j])
        soc_label[i].append(soc_y[i+2][0])
    return test,soc_label

In [11]:
def RNN_train(year,user,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func):
    a,soc_label = generate_train(year,user)
    train_loss=[]
    lr_list = []
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50,100,150,200,250,300], gamma=0.6)
    for step in range(N_EPOCHS):
        error=0.0
        for batch_num in range(batch_nums):#此处分成了43个batch 
            x=torch.from_numpy(np.array(a[:,batch_num,np.newaxis])).float().to(device)
            y=torch.from_numpy(np.array(soc_label[batch_num])[:,np.newaxis,np.newaxis]).float().to(device)
            prediction, h_state = rnn(x, h_state)  # RNN输出（预测结果，隐藏状态）
            h_state = h_state.detach()  # 这一行很重要，将每一次输出的中间状态传递下去(不带梯度)，将h_state分离出来，同时不影响后向计算
            loss = loss_func(prediction, y)
            error += loss
            optimizer.zero_grad()#清零，模型的参数梯度设成0
            loss.backward()
            optimizer.step()
            scheduler.step()  # 学习率更新
            lr_list.append(optimizer.state_dict()['param_groups'][0]['lr'])
            #torch.save(rnn.state_dict(), 'C:/Users/chenxihui/Desktop/code/project/rnn.pt')
        error /= batch_nums
        train_loss.append(error)
    print(str(user)+"  train Loss = " + str(error))
    if not os.path.exists("./model/"):
        os.makedirs("./model/")
    filepath = "./model/rnn"+str(year)+"-"+str(user)+".model"
    torch.save({"model_state_dict" : rnn.state_dict(),
            "optimizer_state_dict" : optimizer.state_dict()},
            filepath)   
    #print("Model has been saved.")


In [12]:
def main():
    ###超参数设置
    year=2012
    TIME_STEP = 48  # RNN时间步长
    INPUT_SIZE = 336  # RNN输入尺寸
    INIT_LR = 0.01  # 初始学习率
    N_EPOCHS = 300  # 训练回数
    Batch_size=1
    batch_nums=301
    train_loss,test_loss=[],[]
    ### RNN建立
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    class RNN(nn.Module):
        def __init__(self):
            super(RNN, self).__init__()
            self.rnn = nn.RNN(
                input_size=INPUT_SIZE,
                hidden_size=20,  # RNN隐藏神经元个数20
                num_layers=1,  # RNN隐藏层个数
            )
            self.out = nn.Linear(20, 1).to(device)#全连接层

        def forward(self, x, h):
            out, h = self.rnn(x, h)
            prediction = self.out(out)
            return prediction, h
        
    rnn = RNN()
    optimizer = torch.optim.Adam(rnn.parameters(), lr=INIT_LR)
    loss_func = nn.MSELoss()
    h_state = None 
    
    ###step 1：得到分类结果，给出属于同一类的用户标号
    position=[1,3,6,12,14,15,16,18,19,22,23,26,28,31,32,34,38,41,42,43,45,46,47]
    scores=[]
    ###step 2：处理该类中MILP、SOC，为了节约时间，将他们先处理保存，之后读取即可
    #for user in position:
        #GG_array,Total_Load_array,pv_size=Read_year(year,user)
        #soc,pg_import,pg_export,pb_c,pb_d=MILP_365(Total_Load_array,GG_array,user)
    ###step 3：对每个用户训练一个模型
    for user in position:
        #RNN_train(2012,user,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func)
        RNN_test(2012,user,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func)
    ###step 4：对每个用户训练出的模型进行测试（在其他用户身上）
        testloss=model_test(year,user,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func)
        scores.append(testloss)
    ###step 5:选出类的特征模型
    minscore=20
    for i in range(len(scores)):
        if(minscore>scores[i]):
            minscore=scores[i]
    for i in range(len(scores)):
        if(minscore==scores[i]):
            print(position[i])#得到该类中的特征模型的用户编号

In [19]:
###超参数设置
year=2010
TIME_STEP = 48  # RNN时间步长
INPUT_SIZE = 336  # RNN输入尺寸
INIT_LR = 0.01  # 初始学习率
N_EPOCHS = 300  # 训练回数
Batch_size=1
batch_nums=301
train_loss,test_loss=[],[]
    ### RNN建立
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(
                input_size=INPUT_SIZE,
                hidden_size=20,  # RNN隐藏神经元个数20
                num_layers=1,  # RNN隐藏层个数
            )
        self.out = nn.Linear(20, 1).to(device)#全连接层

    def forward(self, x, h):
            out, h = self.rnn(x, h)
            prediction = self.out(out)
            return prediction, h
        
rnn = RNN()
optimizer = torch.optim.Adam(rnn.parameters(), lr=INIT_LR)
loss_func = nn.MSELoss()
h_state = None 
    
    ###step 1：得到分类结果，给出属于同一类的用户标号
position=[12,13,21]
scores=[]
    ###step 2：处理该类中MILP、SOC，为了节约时间，将他们先处理保存，之后读取即可
#for user in position:
    #GG_array,Total_Load_array,pv_size=Read_year(year,user)
    #soc,pg_import,pg_export,pb_c,pb_d=MILP_365(Total_Load_array,GG_array,user)
    ###step 3：对每个用户训练一个模型
for user in position:
    #RNN_train(2012,user,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func)
    RNN_test(year,user,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func)
    ###step 4：对每个用户训练出的模型进行测试（在其他用户身上）
    testloss=model_test(year,user,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func)
    print(str(user)+":"+str(testloss))
    scores.append(testloss)
    

Testing...load model...


UnpicklingError: A load persistent id instruction was encountered,
but no persistent_load function was specified.

NameError: name 'scores' is not defined

In [12]:
###step 4：对每个用户训练出的模型进行测试（在其他用户身上）
def model_test(year,user,rnn,optimizer,N_EPOCHS,batch_nums,device,h_state,loss_func):    
    position=[1,3,6,12,14,15,16,18,19,22,23,26,28,31,32,34,38,41,42,43,45,46,47]
    print("Testing...load model...")
    filepath = "./model/rnn"+str(year)+"-"+str(user)+".model"
    checkpoint = torch.load(filepath)
    rnn.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

    rnn.eval()
    testloss=0
    for num in position:
        test,soc_label=generate_test(2012,num)
        with torch.no_grad():
            for step in range(40):
                test_x=torch.from_numpy(np.array(test[:,step,np.newaxis])).float().to(device)
                test_y = torch.from_numpy(np.array(soc_label[step+301])[:,np.newaxis,np.newaxis]).float().to(device)
                predict, h_state = rnn(test_x,h_state)
                loss_test = loss_func(predict, test_y)
                testloss+=loss_test
    testloss=testloss/(40*23)
    print("Test has been done! The LOSS is: "+str(testloss))
    return testloss

In [16]:
model_test(2012,6)

Testing...load model...
Test has been done in! The LOSS is: tensor(1.00000e-03 *
       1.4796)


In [17]:
year=2012
    ###step 1：得到分类结果，给出属于同一类的用户标号
position=[1,3,6,12,14,15,16,18,19,22,23,26,28,31,32,34,38,41,42,43,45,46,47]
    ###step 2：处理该类中MILP、SOC，为了节约时间，将他们先处理保存，之后读取即可
for user in position:
    GG_array,Total_Load_array,pv_size=Read_year(year,user)
    soc,pg_import,pg_export,pb_c,pb_d=MILP_365(Total_Load_array,GG_array,user)
    save(soc,"C:/Users/chenxihui/Desktop/code/project/processed_data/soc-"+str(year)+"-"+str(user)+"-"+".xlsx")
    print(str(user)+"has been done!!")

KeyboardInterrupt: 

In [12]:
year=2012

users=['2','13','14','20','33','35','38','39','56','69','73','74','75'
          ,'82','87','88','101','104','106','109','110','119','124','130'
          ,'137','141','144','152','153','157','161','169','176','184','188'
          ,'189','193','201','202','204','206','207','210','211','212','214'
          ,'218','244','246','253','256','273','276','297']
pv=[]
for i in range(54): 
    GG_array,Total_Load_array,pv_size=Read_year(year,i)
    pv.append(pv_size)

a={}
for i in range(54):
    a[i]= pv[i]
L=a
#L=list(a.items())
#L.sort(key=lambda x:x[1],reverse=False)
for i in range(54):
    if(i in [6,7,33,38,22]):
        L[i]=3
    elif(i in [2,17 ,34 ,18,23,26,43,53 ,45,46,48,3,39,0,32]):
        L[i]=4
    elif(i in [37,15 ,24 ,40 ,42,10,14 ,47 ,52,9 ,16 ,31,35 ,4 ,5,8 ,21,28,44,1]):
        L[i]=5
    elif(i in [41,20,49,25]):
        L[i]=6
    elif(i in [51,13] ):
        L[i]=7
    elif(i in [36,11] ):
        L[i]=8
    elif(i in [27,50] ):
        L[i]=9
    elif(i in [19,12]):
        L[i]=10

In [30]:
L={0: 4,1: 5,2: 4,3: 4,4: 5,5: 5,6: 3,7: 3,8: 5,9: 5,10: 5,11: 8,12: 10,13: 7,14: 5,
 15: 5,16: 5,17: 4,18: 4,19: 10,20: 6,21: 5,22: 3,23: 4,24: 5,25: 6,26: 4,27: 9,28: 5,29: 9.99,
 30: 3.2,31: 5,32: 4,33: 3,34: 4,35: 5,36: 8,37: 5,38: 3,39: 4,40: 5,41: 6,42: 5,43: 4,
 44: 5,45: 4,46: 4,47: 5,48: 4,49: 6,50: 9,51: 7,52: 5,53: 4}

In [25]:
#因为在MILP时，PVsize大小啥的都是变的
year=2012

users=['2','13','14','20','33','35','38','39','56','69','73','74','75'
          ,'82','87','88','101','104','106','109','110','119','124','130'
          ,'137','141','144','152','153','157','161','169','176','184','188'
          ,'189','193','201','202','204','206','207','210','211','212','214'
          ,'218','244','246','253','256','273','276','297']
position=[1,3,6,12,14,15,16,18,19,22,23,26,28,31,32,34,38,41,42,43,45,46,47]
pv=[]
for item in position: 
    GG_array,Total_Load_array,pv_size=Read_year(year,item)
    #soc,pg_import,pg_export,pb_c,pb_d=MILP_365(pv_size,Total_Load_array,GG_array)
    #save(soc,"C:/Users/chenxihui/Desktop/code/project/processed_data/soc"+str(year)+str(i)+".xlsx")
    #print(str(i)+"has been done!")
    #print(pv_size)
    pv.append(pv_size)

2.22
1.57
1.05
8.0
2.0
1.8
2.04
1.5
6.2
1.2
1.5
1.5
2.1
2.04
1.75
1.48
1.05
2.7
1.94
1.5
1.53
1.53
2.0
